# Transport Network Metrics

## 0 Setup environment

In [1]:
import os

import geopandas as gpd
import networkx as nx
import osmnx as ox
import pandas as pd
from networkx import Graph

In [2]:
BASE_PATH = "./data"
BOUNDARIES_FILE = os.path.join(BASE_PATH, "boundaries.gpkg")
TRANSPORT_NETWORK_FILE = os.path.join(BASE_PATH, "merged_network_simplified.graphml")

## 1 Load town boundaries

In [3]:
gdf_boundaries = gpd.read_file(BOUNDARIES_FILE)
gdf_boundaries.head()

,town,province,geometry
0,Bacacay,Albay,"MULTIPOLYGON (((123.84193 13.3341, 123.84204 1..."
1,Camalig,Albay,"MULTIPOLYGON (((123.6559 13.06131, 123.65536 1..."
2,Daraga,Albay,"MULTIPOLYGON (((123.71487 13.03995, 123.71474 ..."
3,Guinobatan,Albay,"MULTIPOLYGON (((123.68355 13.25321, 123.67729 ..."
4,Jovellar,Albay,"MULTIPOLYGON (((123.6559 13.06131, 123.65568 1..."


## 2 Load transport network

In [4]:
graph_transport = ox.load_graphml(TRANSPORT_NETWORK_FILE)
gdf_nodes = ox.graph_to_gdfs(graph_transport, edges=False)
gdf_nodes.head()

,y,x,street_count,mode,highway,junction,railway,geometry
osmid,,,,,,,,
300744370,11.762615,124.033762,3,road,NaN,NaN,NaN,POINT (124.03376 11.76261)
300744933,11.764680,124.063952,3,road,NaN,NaN,NaN,POINT (124.06395 11.76468)
300744970,11.863829,124.057783,3,road,NaN,NaN,NaN,POINT (124.05778 11.86383)
300745522,11.910286,123.908284,3,road,NaN,NaN,NaN,POINT (123.90828 11.91029)
300746507,11.964546,123.991640,3,road,NaN,NaN,NaN,POINT (123.99164 11.96455)


## 3 Compute centrality measures for all nodes

In [5]:
filepath = os.path.join(BASE_PATH, "centrality_measures.csv")

In [6]:
gdf_nodes["degree"] = gdf_nodes.index.map(nx.degree_centrality(graph_transport))
gdf_nodes[["degree"]].to_csv(filepath)
gdf_nodes["degree"].describe()

count    56932.000000
mean         0.000044
std          0.000019
min          0.000018
25%          0.000018
50%          0.000053
75%          0.000053
max          0.000474
Name: degree, dtype: float64

In [7]:
gdf_nodes["closeness"] = gdf_nodes.index.map(nx.closeness_centrality(graph_transport))
gdf_nodes[["degree", "closeness"]].to_csv(filepath)
gdf_nodes["closeness"].describe()

count    56932.000000
mean         0.004952
std          0.001122
min          0.000018
25%          0.004071
50%          0.005072
75%          0.005756
max          0.007339
Name: closeness, dtype: float64

In [8]:
gdf_nodes["betweenness"] = gdf_nodes.index.map(nx.betweenness_centrality(graph_transport))
gdf_nodes[["degree", "closeness", "betweenness"]].to_csv(filepath)
gdf_nodes["betweenness"].describe()

count    56932.000000
mean         0.003582
std          0.019033
min          0.000000
25%          0.000000
50%          0.000070
75%          0.000460
max          0.390183
Name: betweenness, dtype: float64

## 4 Compute average metrics for each town

In [12]:
def calculate_town_metrics(
    town_name: str,
    boundaries_gdf: gpd.GeoDataFrame,
    nodes_gdf: gpd.GeoDataFrame,
    full_graph: nx.Graph,
) -> dict:
    """Calculates all transport network metrics for a single town."""
    town_metrics = {"town": town_name}
    town_boundary = boundaries_gdf[boundaries_gdf["town"] == town_name]

    nodes_in_town = gpd.sjoin(nodes_gdf, town_boundary, how="inner", predicate="within")
    town_metrics["avg_degree"] = nodes_in_town["degree"].mean()
    town_metrics["avg_betweenness"] = nodes_in_town["betweenness"].mean()
    town_metrics["avg_closeness"] = nodes_in_town["closeness"].mean()

    town_subgraph = full_graph.subgraph(nodes_in_town.index)

    # if not nx.is_connected(town_subgraph):
    #     largest_cc = max(nx.connected_components(town_subgraph), key=len)
    #     subgraph_for_analysis = town_subgraph.subgraph(largest_cc)
    # else:
    #     subgraph_for_analysis = town_subgraph

    town_metrics["network_density"] = nx.density(town_subgraph)
    town_metrics["avg_clustering"] = nx.average_clustering(Graph(town_subgraph))
    # town_metrics["network_diameter"] = nx.diameter(subgraph_for_analysis)
    # town_metrics["avg_path_length"] = nx.average_shortest_path_length(subgraph_for_analysis)

    return town_metrics

In [13]:
all_town_metrics = [
    calculate_town_metrics(town_name, gdf_boundaries, gdf_nodes.copy(), graph_transport)
    for town_name in sorted(gdf_boundaries["town"].unique())
]

df_town_metrics = pd.DataFrame(all_town_metrics).set_index("town")
df_town_metrics.head()

,avg_degree,avg_betweenness,avg_closeness,network_density,avg_clustering
town,,,,,
Aroroy,0.000043,0.001804,0.005285,0.003937,0.042272
Baao,0.000043,0.001849,0.005981,0.004325,0.025000
Bacacay,0.000042,0.001171,0.004564,0.004248,0.020644
Bagamanoc,0.000042,0.000905,0.003034,0.014739,0.056911
Balatan,0.000045,0.013441,0.006532,0.013931,0.038251


In [14]:
filepath = os.path.join(BASE_PATH, "town_metrics.csv")
df_town_metrics.to_csv(filepath, index=True)